In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
from scipy.interpolate import interp1d
from tqdm import tqdm

class SchwarzschildRayTracerBatch:
    def __init__(self, bh_mass=1.0, device=None):
        self.G = 1.0
        self.c = 1.0
        self.E = 1.0
        self.M = bh_mass
        self.rs = 2 * self.G * self.M / self.c**2
        self.b_max = 15 * self.rs
        self.device = device if device else (torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu'))

    def V(self, r, b):
        return (b**2 / (2 * r**2) - self.M * b**2 / r**3)

    def dV_dr(self, r, b):
        return (-b**2 / r**3 + 3 * self.M * b**2 / r**4)

    def system(self, y, b):
        r = y[:, 0]
        phi = y[:, 1]
        dr_dtau = y[:, 2]

        d2r_dtau2 = -self.dV_dr(r, b)
        dphi_dtau = b / r**2

        dydtau = torch.stack([dr_dtau, dphi_dtau, d2r_dtau2], dim=1)
        return dydtau

    def rkf45_step_batch(self, f, t, y, h, b):
        a = [0, 1/4, 3/8, 12/13, 1, 1/2]
        b_ = [
            [],
            [1/4],
            [3/32, 9/32],
            [1932/2197, -7200/2197, 7296/2197],
            [439/216, -8, 3680/513, -845/4104],
            [-8/27, 2, -3544/2565, 1859/4104, -11/40]
        ]
        c4 = [25/216, 0, 1408/2565, 2197/4104, -1/5, 0]
        c5 = [16/135, 0, 6656/12825, 28561/56430, -9/50, 2/55]

        k1 = f(y, b)
        k2 = f(y + h * (b_[1][0] * k1), b)
        k3 = f(y + h * (b_[2][0] * k1 + b_[2][1] * k2), b)
        k4 = f(y + h * (b_[3][0] * k1 + b_[3][1] * k2 + b_[3][2] * k3), b)
        k5 = f(y + h * (b_[4][0] * k1 + b_[4][1] * k2 + b_[4][2] * k3 + b_[4][3] * k4), b)
        k6 = f(y + h * (b_[5][0] * k1 + b_[5][1] * k2 + b_[5][2] * k3 + b_[5][3] * k4 + b_[5][4] * k5), b)

        y4 = y + h * (c4[0]*k1 + c4[2]*k3 + c4[3]*k4 + c4[4]*k5)
        y5 = y + h * (c5[0]*k1 + c5[2]*k3 + c5[3]*k4 + c5[4]*k5 + c5[5]*k6)

        error = torch.norm(y5 - y4, dim=1)
        return y5, error

    def solve_batch(self, b_array, r0=100.0, phi0=0.0, tau_span=(0, 140), h=0.1, tol=1e-5, N=1000):
        b = torch.tensor(b_array, dtype=torch.float32, device=self.device)
        batch_size = b.shape[0]
        r0_t = torch.full((batch_size,), r0, dtype=torch.float32, device=self.device)
        phi0_t = torch.full((batch_size,), phi0, dtype=torch.float32, device=self.device)

        V0 = self.V(r0_t, b)
        dr0 = -torch.sqrt(torch.clamp(self.E**2 - 2 * V0, min=0.0))

        y = torch.stack([r0_t, phi0_t, dr0], dim=1)
        t = tau_span[0]
        tf = tau_span[1]

        ys = [y.cpu()]
        active = torch.ones(batch_size, dtype=torch.bool, device=self.device)

        m=int((tf-t)/h/N)
        n=0

        with tqdm(total=N, desc="Renderizando", unit="px") as pbar:
          while t < tf and active.any():
            y_next, error = self.rkf45_step_batch(self.system, t, y, h, b)
            y = torch.where(active.unsqueeze(1), y_next, y)
            active = active & (y[:, 0] > self.rs)
            n +=1
            if n==m:
              ys.append(y.cpu())
              n=0
              pbar.update(1)
            t += h

        return torch.stack(ys)

    def render(self, N_x=100, N_y=100, gamma_deg=20, save_path=None):
        ancho, alto = N_x, N_y
        imagen = Image.new('RGB', (ancho, alto), color=(0,0,0))
        pixeles = imagen.load()

        R= self.matriz_rotacion(0,0, gamma_deg)

        xs = np.linspace(-1, 1, ancho)
        ys = np.linspace(-1, 1, alto)
        xv, yv = np.meshgrid(xs, ys)

        b_vals = self.b_max * np.sqrt(xv.flatten()**2 + yv.flatten()**2)
        N=400

        sol_batch = self.solve_batch(b_vals, r0=100*self.rs, tau_span=(0,2000), h=0.1, N=N)
        rs = self.rs

        r_vals = sol_batch[:, :, 0].numpy()
        phi_vals = sol_batch[:, :, 1].numpy()

        x1 =np.zeros(N)
        y1 =np.zeros(N)
        z1 =np.zeros(N)

        idx = 0
        with tqdm(total=ancho, desc="Renderizando", unit="px") as pbar:
          for i in range(ancho):
              for j in range(alto):
                  b = self.b_max * np.sqrt(xv[j,i]**2 + yv[j,i]**2)

                  r_traj = r_vals[:, idx]
                  phi_traj = phi_vals[:, idx]
                  #print(len(r_traj))

                  theta = np.arctan2(yv[j,i], xv[j,i])

                  x = r_traj*np.cos(theta) * np.sin(phi_traj)
                  y = r_traj*np.sin(theta) * np.sin(phi_traj)
                  z = r_traj*np.cos(phi_traj)

                  tau_original = np.linspace(0, len(r_traj)-1, len(r_traj))  # o el vector real de tau si tienes

                  r2 = np.sqrt(x**2 + y**2 + z**2)

                  indices_validos = np.where((r2 > 6) & (r2 < 20))[0]

                  if len(indices_validos) > 1:

                    tau_sub = tau_original[indices_validos]
                    r_sub = r_traj[indices_validos]
                    phi_sub = phi_traj[indices_validos]

                    if len(tau_sub) >= 4:  # mínimo 4 puntos para 'cubic'
                        interp_r = interp1d(tau_sub, r_sub, kind='cubic')
                        interp_phi = interp1d(tau_sub, phi_sub, kind='cubic')
                    else:
                        interp_r = interp1d(tau_sub, r_sub, kind='linear')
                        interp_phi = interp1d(tau_sub, phi_sub, kind='linear')

                    tau_fino = np.linspace(tau_sub[0], tau_sub[-1], num=300)

                    r_interp = interp_r(tau_fino)
                    phi_interp = interp_phi(tau_fino)

                    # Calcular coordenadas 3D interpoladas
                    x_interp = r_interp * np.cos(theta) * np.sin(phi_interp)
                    y_interp = r_interp * np.sin(theta) * np.sin(phi_interp)
                    z_interp = r_interp * np.cos(phi_interp)

                    points = np.vstack((x_interp, y_interp, z_interp)).T  # shape (N, 3)
                    rotated_points = points @ R.T  # shape (N, 3)

                    x1 = rotated_points[:, 0]
                    y1 = rotated_points[:, 1]
                    z1 = rotated_points[:, 2]

                    A = (np.sqrt(x1**2 + z1**2) > 8.62) & (np.sqrt(x1**2 + z1**2) < 18.32)
                    B = np.abs(np.arctan(y1 / np.sqrt(x1**2 + z1**2))) < 0.03

                    if np.any(A & B):
                        pixeles[i, j] = (255, 128, 0)

                    else:
                        pixeles[i, j] = (0, 0, 0)
                  else:
                    pixeles[i, j] = (0, 0, 0)

                  idx += 1
              pbar.update(1)

        if save_path:
            imagen.save(save_path)
        else:
          imagen.save("Multimedia/Imagen_disco_de_acreción.png")

    def matriz_rotacion(self, theta_deg=0, phi_deg=0, gamma_deg=0):
        theta = np.deg2rad(theta_deg)
        phi = np.deg2rad(phi_deg)
        gamma= np.deg2rad(gamma_deg)
        Rz = np.array([
            [np.cos(theta), -np.sin(theta), 0],
            [np.sin(theta),  np.cos(theta), 0],
            [0, 0, 1]
        ])
        Ry = np.array([
            [np.cos(phi), 0, np.sin(phi)],
            [0, 1, 0],
            [-np.sin(phi), 0, np.cos(phi)]
        ])
        Rx = np.array([
            [1, 0, 0],
            [0, np.cos(gamma), np.sin(gamma)],
            [0, -np.sin(gamma), np.cos(gamma)]
        ])
        return Rz @ Ry @ Rx

rt_batch = SchwarzschildRayTracerBatch(bh_mass=1.0)
rt_batch.render(N_x=100, N_y=100)

Renderizando: 100%|██████████| 100/100 [00:02<00:00, 40.00px/s]


In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
from scipy.interpolate import interp1d
from tqdm import tqdm
import imageio

class SchwarzschildRayTracerBatch:
    def __init__(self, bh_mass=1.0, device=None):
        self.G = 1.0
        self.c = 1.0
        self.E = 1.0
        self.M = bh_mass
        self.rs = 2 * self.G * self.M / self.c**2
        self.b_max = 15 * self.rs
        self.device = device if device else (torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu'))

    def V(self, r, b):
        return (b**2 / (2 * r**2) - self.M * b**2 / r**3)

    def dV_dr(self, r, b):
        return (-b**2 / r**3 + 3 * self.M * b**2 / r**4)

    def system(self, y, b):
        r = y[:, 0]
        phi = y[:, 1]
        dr_dtau = y[:, 2]

        d2r_dtau2 = -self.dV_dr(r, b)
        dphi_dtau = b / r**2

        dydtau = torch.stack([dr_dtau, dphi_dtau, d2r_dtau2], dim=1)
        return dydtau

    def rkf45_step_batch(self, f, t, y, h, b):
        a = [0, 1/4, 3/8, 12/13, 1, 1/2]
        b_ = [
            [],
            [1/4],
            [3/32, 9/32],
            [1932/2197, -7200/2197, 7296/2197],
            [439/216, -8, 3680/513, -845/4104],
            [-8/27, 2, -3544/2565, 1859/4104, -11/40]
        ]
        c4 = [25/216, 0, 1408/2565, 2197/4104, -1/5, 0]
        c5 = [16/135, 0, 6656/12825, 28561/56430, -9/50, 2/55]

        k1 = f(y, b)
        k2 = f(y + h * (b_[1][0] * k1), b)
        k3 = f(y + h * (b_[2][0] * k1 + b_[2][1] * k2), b)
        k4 = f(y + h * (b_[3][0] * k1 + b_[3][1] * k2 + b_[3][2] * k3), b)
        k5 = f(y + h * (b_[4][0] * k1 + b_[4][1] * k2 + b_[4][2] * k3 + b_[4][3] * k4), b)
        k6 = f(y + h * (b_[5][0] * k1 + b_[5][1] * k2 + b_[5][2] * k3 + b_[5][3] * k4 + b_[5][4] * k5), b)

        y4 = y + h * (c4[0]*k1 + c4[2]*k3 + c4[3]*k4 + c4[4]*k5)
        y5 = y + h * (c5[0]*k1 + c5[2]*k3 + c5[3]*k4 + c5[4]*k5 + c5[5]*k6)

        error = torch.norm(y5 - y4, dim=1)
        return y5, error

    def solve_batch(self, b_array, r0=100.0, phi0=0.0, tau_span=(0, 140), h=0.1, tol=1e-5, N=1000):
        b = torch.tensor(b_array, dtype=torch.float32, device=self.device)
        batch_size = b.shape[0]
        r0_t = torch.full((batch_size,), r0, dtype=torch.float32, device=self.device)
        phi0_t = torch.full((batch_size,), phi0, dtype=torch.float32, device=self.device)

        V0 = self.V(r0_t, b)
        dr0 = -torch.sqrt(torch.clamp(self.E**2 - 2 * V0, min=0.0))

        y = torch.stack([r0_t, phi0_t, dr0], dim=1)
        t = tau_span[0]
        tf = tau_span[1]

        ys = [y.cpu()]
        active = torch.ones(batch_size, dtype=torch.bool, device=self.device)

        m=int((tf-t)/h/N)
        n=0
        with tqdm(total=N, desc="Renderizando", unit="px") as pbar:
          while t < tf and active.any():
            #while error>
            y_next, error = self.rkf45_step_batch(self.system, t, y, h, b)

            y = torch.where(active.unsqueeze(1), y_next, y)
            active = active & (y[:, 0] > self.rs)
            n +=1
            if n==m:
              ys.append(y.cpu())
              n=0
              pbar.update(1)
            t += h

        return torch.stack(ys)

    def render(self, N_x=100, N_y=100, gamma_deg=20):
        ancho, alto = N_x, N_y
        imagen = Image.new('RGB', (ancho, alto), color=(0,0,0))
        imagen2 = Image.new('RGB', (ancho, alto), color=(0,0,0))
        pixeles = imagen.load()
        pixeles2= imagen2.load()

        R= self.matriz_rotacion(0,0, gamma_deg)

        xs = np.linspace(-1, 1, ancho)
        ys = np.linspace(-1, 1, alto)
        xv, yv = np.meshgrid(xs, ys)

        b_vals = self.b_max * np.sqrt(xv.flatten()**2 + yv.flatten()**2)
        N=400

        sol_batch = self.solve_batch(b_vals, r0=100*self.rs, tau_span=(0,2000), h=0.1, N=N)
        rs = self.rs

        r_vals = sol_batch[:, :, 0].numpy()
        phi_vals = sol_batch[:, :, 1].numpy()

        x1 =np.zeros(N)
        y1 =np.zeros(N)
        z1 =np.zeros(N)

        gamma_deg_vals=np.linspace(0,180, 36)
        with tqdm(total=ancho*len(gamma_deg_vals), desc="Renderizando", unit="px") as pbar:
          with imageio.get_writer("rotacion.mp4", fps=10) as writer:
            for gamma_deg in gamma_deg_vals:
              R= self.matriz_rotacion(0,0, gamma_deg)
              idx = 0
              for i in range(ancho):
                  for j in range(alto):
                      b = self.b_max * np.sqrt(xv[j,i]**2 + yv[j,i]**2)

                      r_traj = r_vals[:, idx]
                      phi_traj = phi_vals[:, idx]
                      #print(len(r_traj))

                      theta = np.arctan2(yv[j,i], xv[j,i])

                      if b<=3*np.sqrt(3)/2*rs:
                        phi = np.interp(rs, r_traj[-2:], phi_traj[-2:])
                        x = rs * np.cos(theta) * np.sin(phi)
                        y = rs * np.sin(theta) * np.sin(phi)
                        z = -rs * np.cos(phi)
                        x1,y1,z1 = np.dot([x,y,z], R)
                        theta1 = np.arccos(y1 / rs)+np.pi/2
                        phi1 = np.arctan2(z1, x1)+np.pi

                        if ((int(phi1 / (2*np.pi) * 12) % 2) == 0) and ((int(theta1 / np.pi * 7) % 2) == 0):
                          color = (0, 0, 255)
                        elif ((int(phi1 / (2*np.pi) * 12 + 1) % 2) == 0) and ((int(theta1 / np.pi * 7 + 1) % 2) == 0):
                          color = (0, 0, 255)
                        else:
                          color = (0, 0, 0)

                        pixeles[i, j] = color

                        if (b<=rs):
                          phi0=np.arcsin(b/self.rs)
                          x = rs * np.cos(theta) * np.sin(phi0)
                          y = rs * np.sin(theta) * np.sin(phi0)
                          z = -rs * np.cos(phi0)
                          x1,y1,z1 = np.dot([x,y,z], R)
                          theta1 = np.arccos(y1 / rs)+np.pi/2
                          phi1 = np.arctan2(z1, x1)+np.pi
                          if ((int(phi1 / (2*np.pi) * 12) % 2) == 0) and ((int(theta1 / np.pi * 7) % 2) == 0):
                              color2 = (255, 0, 0)
                          elif ((int(phi1 / (2*np.pi) * 12 + 1) % 2) == 0) and ((int(theta1 / np.pi * 7 + 1) % 2) == 0):
                              color2 = (255, 0, 0)
                          else:
                            color2 = (0, 0, 0)

                          pixeles2[i, j] = color2

                      x = r_traj*np.cos(theta) * np.sin(phi_traj)
                      y = r_traj*np.sin(theta) * np.sin(phi_traj)
                      z = r_traj*np.cos(phi_traj)

                      tau_original = np.linspace(0, len(r_traj)-1, len(r_traj))  # o el vector real de tau si tienes

                      r = np.sqrt(x**2 + y**2 + z**2)

                      indices_validos = np.where((r > 7) & (r < 20))[0]

                      if len(indices_validos) > 1:

                        tau_sub = tau_original[indices_validos]
                        r_sub = r_traj[indices_validos]
                        phi_sub = phi_traj[indices_validos]

                        if len(tau_sub) >= 4:  # mínimo 4 puntos para 'cubic'
                            interp_r = interp1d(tau_sub, r_sub, kind='cubic')
                            interp_phi = interp1d(tau_sub, phi_sub, kind='cubic')
                        else:
                            interp_r = interp1d(tau_sub, r_sub, kind='linear')
                            interp_phi = interp1d(tau_sub, phi_sub, kind='linear')

                        tau_fino = np.linspace(tau_sub[0], tau_sub[-1], num=300)

                        r_interp = interp_r(tau_fino)
                        phi_interp = interp_phi(tau_fino)

                        # Calcular coordenadas 3D interpoladas
                        x_interp = r_interp * np.cos(theta) * np.sin(phi_interp)
                        y_interp = r_interp * np.sin(theta) * np.sin(phi_interp)
                        z_interp = r_interp * np.cos(phi_interp)

                        points = np.vstack((x_interp, y_interp, z_interp)).T  # shape (N, 3)
                        rotated_points = points @ R.T  # shape (N, 3)

                        x1 = rotated_points[:, 0]
                        y1 = rotated_points[:, 1]
                        z1 = rotated_points[:, 2]

                        A = (np.sqrt(x1**2 + z1**2) > 8.62) & (np.sqrt(x1**2 + z1**2) < 18.314)
                        B = np.abs(np.arctan(y1 / np.sqrt(x1**2 + z1**2))) < 0.03

                        if np.any(A & B):
                            pixeles[i, j] = (255, 128, 0)

                        else:
                            pixeles[i, j] = (0, 0, 0)

                      idx += 1
                  pbar.update(1)
              writer.append_data(imageio.imread(imagen))


    def matriz_rotacion(self, theta_deg=0, phi_deg=0, gamma_deg=0):
        theta = np.deg2rad(theta_deg)
        phi = np.deg2rad(phi_deg)
        gamma= np.deg2rad(gamma_deg)
        Rz = np.array([
            [np.cos(theta), -np.sin(theta), 0],
            [np.sin(theta),  np.cos(theta), 0],
            [0, 0, 1]
        ])
        Ry = np.array([
            [np.cos(phi), 0, np.sin(phi)],
            [0, 1, 0],
            [-np.sin(phi), 0, np.cos(phi)]
        ])
        Rx = np.array([
            [1, 0, 0],
            [0, np.cos(gamma), np.sin(gamma)],
            [0, -np.sin(gamma), np.cos(gamma)]
        ])
        return Rz @ Ry @ Rx

rt_batch = SchwarzschildRayTracerBatch(bh_mass=1.0)
rt_batch.render(N_x=100, N_y=100)

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 225)

In [12]:
import cupy as cp
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
from scipy.interpolate import interp1d
from tqdm import tqdm
import imageio

class SchwarzschildRayTracerBatch:
    def __init__(self, bh_mass=1.0, device=None):
        self.G = 1.0
        self.c = 1.0
        self.E = 1.0
        self.M = bh_mass
        self.rs = 2 * self.G * self.M / self.c**2
        self.b_max = 15 * self.rs
        self.device = device if device else (torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu'))

    def V(self, r, b):
        return (b**2 / (2 * r**2) - self.M * b**2 / r**3)

    def dV_dr(self, r, b):
        return (-b**2 / r**3 + 3 * self.M * b**2 / r**4)

    def system(self, y, b):
        r = y[:, 0]
        phi = y[:, 1]
        dr_dtau = y[:, 2]

        d2r_dtau2 = -self.dV_dr(r, b)
        dphi_dtau = b / r**2

        dydtau = torch.stack([dr_dtau, dphi_dtau, d2r_dtau2], dim=1)
        return dydtau

    def rkf45_step_batch(self, f, t, y, h, b):
        a = [0, 1/4, 3/8, 12/13, 1, 1/2]
        b_ = [
            [],
            [1/4],
            [3/32, 9/32],
            [1932/2197, -7200/2197, 7296/2197],
            [439/216, -8, 3680/513, -845/4104],
            [-8/27, 2, -3544/2565, 1859/4104, -11/40]
        ]
        c4 = [25/216, 0, 1408/2565, 2197/4104, -1/5, 0]
        c5 = [16/135, 0, 6656/12825, 28561/56430, -9/50, 2/55]

        k1 = f(y, b)
        k2 = f(y + h * (b_[1][0] * k1), b)
        k3 = f(y + h * (b_[2][0] * k1 + b_[2][1] * k2), b)
        k4 = f(y + h * (b_[3][0] * k1 + b_[3][1] * k2 + b_[3][2] * k3), b)
        k5 = f(y + h * (b_[4][0] * k1 + b_[4][1] * k2 + b_[4][2] * k3 + b_[4][3] * k4), b)
        k6 = f(y + h * (b_[5][0] * k1 + b_[5][1] * k2 + b_[5][2] * k3 + b_[5][3] * k4 + b_[5][4] * k5), b)

        y4 = y + h * (c4[0]*k1 + c4[2]*k3 + c4[3]*k4 + c4[4]*k5)
        y5 = y + h * (c5[0]*k1 + c5[2]*k3 + c5[3]*k4 + c5[4]*k5 + c5[5]*k6)

        error = torch.norm(y5 - y4, dim=1)
        return y5, error

    def solve_batch(self, b_array, r0=100.0, phi0=0.0, tau_span=(0, 140), h=0.1, tol=1e-5, N=1000):
        b = torch.tensor(b_array, dtype=torch.float32, device=self.device)
        batch_size = b.shape[0]
        r0_t = torch.full((batch_size,), r0, dtype=torch.float32, device=self.device)
        phi0_t = torch.full((batch_size,), phi0, dtype=torch.float32, device=self.device)

        V0 = self.V(r0_t, b)
        dr0 = -torch.sqrt(torch.clamp(self.E**2 - 2 * V0, min=0.0))

        y = torch.stack([r0_t, phi0_t, dr0], dim=1)
        t = tau_span[0]
        tf = tau_span[1]

        ys = [y.cpu()]
        active = torch.ones(batch_size, dtype=torch.bool, device=self.device)

        m=int((tf-t)/h/N)
        n=0
        with tqdm(total=N, desc="Calculando trayectorias", unit="Tr") as pbar:
          while t < tf and active.any():
            #while error>
            y_next, error = self.rkf45_step_batch(self.system, t, y, h, b)

            y = torch.where(active.unsqueeze(1), y_next, y)
            active = active & (y[:, 0] > self.rs)
            n +=1
            if n==m:
              ys.append(y.cpu())
              n=0
              pbar.update(1)
            t += h

        return torch.stack(ys)

    def render(self, N_x=100, N_y=100, gamma_deg=20):
        ancho, alto = N_x, N_y

        xs = np.linspace(-1, 1, ancho)
        ys = np.linspace(-1, 1, alto)
        xv, yv = np.meshgrid(xs, ys)

        b_vals = self.b_max * np.sqrt(xv.flatten()**2 + yv.flatten()**2)
        N=500

        sol_batch = self.solve_batch(b_vals, r0=100*self.rs, tau_span=(0,2000), h=0.1, N=N)
        rs = self.rs

        r_vals =  sol_batch[:, :, 0].numpy()
        phi_vals =  sol_batch[:, :, 1].numpy()

        x1 =np.zeros(N)
        y1 =np.zeros(N)
        z1 =np.zeros(N)

        gamma_deg_vals=np.linspace(0,180, 36)
        with tqdm(total=len(gamma_deg_vals)*ancho, desc="Renderizando", unit="Imágenes") as pbar:
          with imageio.get_writer("Multimedia/rotacion.mp4", fps=10) as writer:
            for gamma_deg in gamma_deg_vals:
                R= self.matriz_rotacion(0,0, gamma_deg)
                imagen = Image.new('RGB', (ancho, alto), color=(0,0,0))
                imagen2 = Image.new('RGB', (ancho, alto), color=(0,0,0))
                pixeles = imagen.load()
                pixeles2= imagen2.load()
                idx=0

                for i in range(ancho):
                    for j in range(alto):
                        b = self.b_max * np.sqrt(xv[j,i]**2 + yv[j,i]**2)

                        r_traj = r_vals[:, idx]
                        phi_traj = phi_vals[:, idx]
                        #print(len(r_traj))

                        theta = np.arctan2(yv[j,i], xv[j,i])

                        x = r_traj*np.cos(theta) * np.sin(phi_traj)
                        y = r_traj*np.sin(theta) * np.sin(phi_traj)
                        z = r_traj*np.cos(phi_traj)

                        tau_original = np.linspace(0, len(r_traj)-1, len(r_traj))  # o el vector real de tau si tienes

                        r = np.sqrt(x**2 + y**2 + z**2)

                        indices_validos = np.where((r > 6) & (r < 20))[0]

                        if len(indices_validos) > 1:

                            tau_sub = tau_original[indices_validos]
                            r_sub = r_traj[indices_validos]
                            phi_sub = phi_traj[indices_validos]

                            if len(tau_sub) >= 4:  # mínimo 4 puntos para 'cubic'
                                interp_r = interp1d(tau_sub, r_sub, kind='cubic')
                                interp_phi = interp1d(tau_sub, phi_sub, kind='cubic')
                            else:
                                interp_r = interp1d(tau_sub, r_sub, kind='linear')
                                interp_phi = interp1d(tau_sub, phi_sub, kind='linear')

                            tau_fino = np.linspace(tau_sub[0], tau_sub[-1], num=300)

                            r_interp = interp_r(tau_fino)
                            phi_interp = interp_phi(tau_fino)

                            # Calcular coordenadas 3D interpoladas
                            x_interp = r_interp * np.cos(theta) * np.sin(phi_interp)
                            y_interp = r_interp * np.sin(theta) * np.sin(phi_interp)
                            z_interp = r_interp * np.cos(phi_interp)

                            points = np.vstack((x_interp, y_interp, z_interp)).T  # shape (N, 3)
                            rotated_points = points @ R.T  # shape (N, 3)

                            x1 = rotated_points[:, 0]
                            y1 = rotated_points[:, 1]
                            z1 = rotated_points[:, 2]

                            Rmin=4.32*rs
                            Rmax=9.16*rs

                            A = (np.sqrt(x1**2 + z1**2) > Rmin) & (np.sqrt(x1**2 + z1**2) < Rmax)
                            B = np.abs(np.arctan(y1 / np.sqrt(x1**2 + z1**2))) < 0.03

                            if np.any(A & B):
                                arg=np.where(A & B)[0][0]
                                x1=x1[arg]
                                y1=y1[arg]
                                z1=z1[arg]
                                
                                r=np.sqrt(x1**2 + z1**2)
                                argumento=(r-Rmin)/(Rmax-Rmin)

                                phi= np.arctan2(z1,x1)

                                f=np.abs(np.cos(argumento*12*np.pi))*(1-argumento)
                                
                                f_dop=Rmin*(1-0.75*np.sin(phi)*np.cos(gamma_deg/180*np.pi))*self.c*np.sqrt(self.rs/r)*(r/(r-rs))
                                
                                Red=int((100*f+155))
                                Green=int(10*f+60)
                                Blue=int(20*f_dop*f)

                                pixeles[i, j] = (Red, Green, Blue) 

                            else:
                                pixeles[i, j] = (0, 0, 0)
                        else:
                            pixeles[i, j] = (0, 0, 0)

                        idx += 1
                    pbar.update(1)
                frame = np.array(imagen)
                writer.append_data(frame)

    def matriz_rotacion(self, theta_deg=0, phi_deg=0, gamma_deg=0):
        theta = np.deg2rad(theta_deg)
        phi = np.deg2rad(phi_deg)
        gamma= np.deg2rad(gamma_deg)
        Rz = np.array([
            [np.cos(theta), -np.sin(theta), 0],
            [np.sin(theta),  np.cos(theta), 0],
            [0, 0, 1]
        ])
        Ry = np.array([
            [np.cos(phi), 0, np.sin(phi)],
            [0, 1, 0],
            [-np.sin(phi), 0, np.cos(phi)]
        ])
        Rx = np.array([
            [1, 0, 0],
            [0, np.cos(gamma), np.sin(gamma)],
            [0, -np.sin(gamma), np.cos(gamma)]
        ])
        return Rz @ Ry @ Rx

rt_batch = SchwarzschildRayTracerBatch(bh_mass=1.0)
rt_batch.render(N_x=112, N_y=112)
#rt_batch.render(N_x=512, N_y=512)

Renderizando: 100%|██████████| 4032/4032 [01:11<00:00, 56.18Imágenes/s] 
